In [2]:
# RSNA Breast Cancer Detection - Auto Threshold Version

# =========================================================
# 1️⃣ Imports
# =========================================================
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_curve, precision_score, recall_score

# =========================================================
# 2️⃣ Device
# =========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================================================
# 3️⃣ Dataset Paths
# =========================================================
DATA_PATH = "/kaggle/input/rsna-breast-cancer-detection"
TRAIN_DIR = os.path.join(DATA_PATH, "train")
VAL_DIR   = os.path.join(DATA_PATH, "val")

# =========================================================
# 4️⃣ Transforms (Improved)
# =========================================================
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform)
val_dataset   = datasets.ImageFolder(VAL_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)

print("Classes:", train_dataset.classes)

# =========================================================
# 5️⃣ Model Definition
# =========================================================
from torchvision.models import resnet18, ResNet18_Weights

class BreastCancerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.fc = nn.Linear(self.model.fc.in_features, 1)

    def forward(self, x):
        return self.model(x)

model = BreastCancerCNN().to(device)

# =========================================================
# 6️⃣ Loss & Optimizer (FP-aware)
# =========================================================
pos_weight = torch.tensor([2.0]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# =========================================================
# 7️⃣ Training Loop
# =========================================================
epochs = 15

for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f}")

# =========================================================
# 8️⃣ Automatic Threshold Selection (ROC)
# =========================================================
model.eval()
y_true = []
y_scores = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        probs = torch.sigmoid(outputs).cpu().numpy()

        y_scores.extend(probs.flatten())
        y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_scores = np.array(y_scores)

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Choose threshold minimizing false positives
optimal_idx = np.argmin(fpr + (1 - tpr))
optimal_threshold = thresholds[optimal_idx]

print(f"\n✅ Automatically selected threshold: {optimal_threshold:.3f}")

# =========================================================
# 9️⃣ Final Validation Metrics
# =========================================================
y_pred = (y_scores >= optimal_threshold).astype(int)

precision = precision_score(y_true, y_pred)
recall    = recall_score(y_true, y_pred)

print(f"Precision (↓ FP): {precision:.4f}")
print(f"Recall (↓ FN):    {recall:.4f}")

# =========================================================
# 🔟 Save Model & Threshold
# =========================================================
torch.save({
    "model_state": model.state_dict(),
    "threshold": optimal_threshold
}, "/kaggle/working/breast_cancer_model_with_threshold.pth")

print("Model + threshold saved!")

Using device: cuda
Classes: ['cancer', 'normal']
Epoch [1/15] Loss: 0.2039
Epoch [2/15] Loss: 0.1914
Epoch [3/15] Loss: 0.1895
Epoch [4/15] Loss: 0.1869
Epoch [5/15] Loss: 0.1836
Epoch [6/15] Loss: 0.1773
Epoch [7/15] Loss: 0.1679
Epoch [8/15] Loss: 0.1549
Epoch [9/15] Loss: 0.1384
Epoch [10/15] Loss: 0.1219
Epoch [11/15] Loss: 0.1085
Epoch [12/15] Loss: 0.0996
Epoch [13/15] Loss: 0.0923
Epoch [14/15] Loss: 0.0899
Epoch [15/15] Loss: 0.0873

✅ Automatically selected threshold: 1.000
Precision (↓ FP): 0.9750
Recall (↓ FN):    0.2391
Model + threshold saved!
